In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
from datetime import timedelta

In [2]:
# Load quarterly price data.
df = pd.read_csv(r'/Users/Colin/Desktop/Research project/Data/data_v2/cusip_list_q.csv')
df['datadate'] = pd.to_datetime(df.datadate, format='%d/%m/%Y')
print("Number of unique CUSIP id's: " + str(df['cusip'].nunique()))

Number of unique CUSIP id's: 12441


In [3]:
# Computing number of months that each stock (=cusip) has been traded. #FIXME: how to get consecutive months?
df_12m = df[['datadate', 'cusip', 'prccq']]
df_12m = df_12m.dropna()
df_12m = df_12m.groupby('cusip')['datadate'].agg(['min','max']).rename(columns={'min':'first','max':'last'}).reset_index()
df_12m[['first', 'last']] = df_12m[['first', 'last']].apply(pd.to_datetime)
df_12m['months_traded'] = ((df_12m['last'] - df_12m['first'])/np.timedelta64(1, 'M'))
df_12m['months_traded']= df_12m['months_traded'].astype(int)
df_12m.head()

,cusip,first,last,months_traded
0,000032102,1970-09-30,1978-03-31,89
1,000165100,1983-09-30,1986-03-31,29
2,000209106,1987-06-30,1993-06-30,72
3,000360206,1991-03-31,2017-09-30,318
4,000361105,1970-02-28,2017-08-31,570


In [4]:
# Quality check.
print("Number of consecutive days that TSLA has been traded:")
df_12m.loc[df_12m['cusip'] == '88160R101']

Number of consecutive days that TSLA has been traded:


,cusip,first,last,months_traded
10096,88160R101,2010-06-30,2017-09-30,87


In [5]:
# Creating list with unique CUSIP id's remaining after 12m filter.
cusip_list_12m = df_12m['cusip'].loc[(df_12m['months_traded'] >= 12)].tolist()
print("Number of unique CUSIP id's after 12m filter: " + str(len(set(cusip_list_12m))))

Number of unique CUSIP id's after 12m filter: 10764


In [6]:
# Creating dataframe for market cap filter.
df_marketcap = df[['datadate', 'cusip', 'prccq', 'cshoq']]
df_marketcap = df_marketcap.loc[df_marketcap['cusip'].isin(cusip_list_12m)]
df_marketcap['datadate'] = (pd.to_datetime(df['datadate']) + timedelta(days=1))


In [7]:
# Loading inflation data.
inflation = pd.read_excel(r'/Users/Colin/Desktop/Research project/Data/Data - Final/Inflation adjustment/CPIAUCNS.xls')
inflation.rename(columns={'Date': 'datadate'}, inplace=True)

# Creating index multiplier.
inflation['CPI_Multiplier'] = inflation['CPIAUCNS'].iloc[-1] / inflation['CPIAUCNS']

In [8]:
# Merging df_marketcap with inflation dataframe and creating CPI multiplier.
df_marketcap = pd.merge(df_marketcap, inflation, how='inner', on='datadate')
df_marketcap['CPIAdjMarketCap'] = df_marketcap['prccq'] * df_marketcap['cshoq'] * df_marketcap['CPI_Multiplier']
df_marketcap['CPIAdjMarketCap_max'] = df_marketcap.groupby(['cusip'])['CPIAdjMarketCap'].transform(max)
df_marketcap.head()


,datadate,cusip,prccq,cshoq,CPIAUCNS,CPI_Multiplier,CPIAdjMarketCap,CPIAdjMarketCap_max
0,1970-04-01,000032102,NaN,2.230,38.5,6.879922,NaN,225.860486
1,1970-04-01,001476100,7.875,4.622,38.5,6.879922,250.417124,250.417124
2,1970-04-01,001670108,9.000,0.857,38.5,6.879922,53.064839,86.014019
3,1970-04-01,001688100,21.500,NaN,38.5,6.879922,NaN,7335.588348
4,1970-04-01,02376R102,26.875,NaN,38.5,6.879922,NaN,42394.446634


In [9]:
# Creating list with unique CUSIP id's remaining after 12m filter.
cusip_list_12m_marketcap = set(df_marketcap['cusip'].loc[(df_marketcap['CPIAdjMarketCap_max'] >= 100)].tolist())
print("Number of unique CUSIP id's after 12m and market cap filter: " + str(len(cusip_list_12m_marketcap)))

Number of unique CUSIP id's after 12m and market cap filter: 9768


In [10]:
# Save the above list as cusip_list.txt
f = open('cusip_list.txt', 'w')
print(*cusip_list_12m_marketcap, sep='\n', file = f)
f.close()